<a href="https://colab.research.google.com/github/sri7683/Hello-world/blob/main/Disseratation_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve


In [ ]:

df_1 = pd.read_csv("/content/drive/MyDrive/config/tables/EMBED_OpenData_clinical.csv")
df2 = pd.read_csv("/content/drive/MyDrive/config/tables/EMBED_OpenData_metadata_reduced.csv")

In [ ]:

df_clinical = df_1[['empi_anon','acc_anon','asses', 'tissueden',
                          'desc','side','path_severity','numfind','total_L_find','total_R_find',
                          'massshape','massmargin','massdens','calcfind','calcdistri','calcnumber']]

df_metadata = df2[['anon_dicom_path','acc_anon','StudyDescription','SeriesDescription','FinalImageType','ImageLateralityFinal','ViewPosition','spot_mag','ROI_coords','num_roi']]


In [ ]:
df_merge = pd.merge(df_metadata, df_clinical, on=['acc_anon'])


print(df_merge.describe().T)
df_merge.head(5)


In [ ]:
df_merge.info()
print(df_merge.isnull().sum())

In [ ]:
df_merge = df_merge.dropna(subset=['path_severity'])
df_merge['path_severity'] = df_merge['path_severity'].astype(int)

In [ ]:
df_merge['path_severity'].unique()
sns.countplot(x="path_severity", data=df_merge)

In [ ]:
df_merge['StudyDescription'].unique()

In [ ]:
df_merge['SeriesDescription'].unique()

In [ ]:
df_merge['asses'].unique()
sns.countplot(x="asses", data=df_merge)


In [ ]:
sns.countplot(x="asses", data=df_clinical)

In [ ]:
# TensorFlow libraries
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# basic libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os

import glob
from glob import glob

In [ ]:
Images = '/content/drive/MyDrive/config/sample_data'
# Trimming and replacing the path
df_merge['anon_dicom_path'] = df_merge['anon_dicom_path'].str.replace("/mnt/NAS2/mammo/anon_dicom/cohort_1", "/content/drive/MyDrive/config/sample_data")


In [ ]:
df_merge.to_csv('/content/drive/MyDrive/config/tables/df_merge.csv', index=False)


In [ ]:
# Reset the index
df_merge = df_merge.reset_index(drop=True)

In [ ]:
# a sample path
df_merge.loc[22723, 'anon_dicom_path']

In [ ]:
#we will now get the list of ROIs for each of these images. ROIs are structured as a list of lists, and each image
# can have 0 to multiple ROIs. We will therefore parse the ROI list to expand it such that each row will contain one ROI.
# If an image has multiple ROIs, this will result in multiple rows for that image in the resultant dataframe

#define function
def separate_roi(df):
    df_list = []
    for ind, row in df.iterrows():
        path = row['anon_dicom_path']
        roi_num = [int(s) for s in re.findall(r'\b\d+\b', row['ROI_coords'])]
        if len(roi_num)==4:
            df_list.append([path, row['ROI_coords'], row['ROI_coords']])
        else:
            count = 0
            roi = []
            for i in roi_num:
                count += 1
                roi.append(i)
                if count%4 == 0:
                    df_list.append([path, row['ROI_coords'], "(("+str(roi[0])+", "+str(roi[1])+", "+str(roi[2])+", "+str(roi[3])+"),)"])
                    roi = []
    df_roi_sep = pd.DataFrame(df_list)
    df_roi_sep.columns = ['anon_dicom_path','ROI_coords','ROI_separated']
    df_cp = df.copy()
    df_cp = df_cp.merge(df_roi_sep, how='left', on=['anon_dicom_path','ROI_coords'])
    return df_cp

In [ ]:
pip install pylibjpeg

In [ ]:
import pandas as pd
import numpy as np
import re
import pydicom
import pylibjpeg
import matplotlib.patches as patches
from matplotlib import pyplot as plt


In [ ]:
def stats(df): #the function displays dataframe size, countings of unique patients and unique exams
    print('Dataframe size: ' + str(df.shape))
    try:
        print('# patients: ' + str(df.empi_anon.nunique()))
    except:
        print('# patients: ' + str(df.empi_anon_x.nunique()))
    print('# exams: ' + str(df.acc_anon.nunique()))

In [ ]:
# To export ROIs, filter the ones with ROIs
df_cancer_ROI = df_merge.loc[df_merge.ROI_coords!='()']
stats(df_cancer_ROI)

# Separate multiple ROIs into individual rows
df_cancer_ROI = separate_roi(df_cancer_ROI)
stats(df_cancer_ROI)

# Export to csv
df_cancer_ROI.to_csv("BR0CancerROIs.csv")

In [ ]:
df_cancer_ROI.ROI_coords

In [ ]:
sample_file_name = "/content/drive/MyDrive/config/sample_data/10000879/1.2.842.113970.3.62.1.56868341.20180426.1095160/1.2.840.113684.2750825173.1524118967.4848.25046.1/1.2.826.0.1.3680043.8.498.12326784267688742524174894068105129442.dcm"

In [ ]:
pip install pydicom pylibjpeg-libjpeg

In [ ]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np

dcm = pydicom.dcmread(sample_file_name)
pixel_data = dcm.pixel_array



In [ ]:
# Read in sample dicom file
ds = pydicom.dcmread(sample_file_name)
arr = ds.pixel_array

# Flip the dicom file left to right, the flipping logic is included in the dcm to png file conversion code
new_np_array = np.copy(arr)
#define the width(w) and height(h) of the image
h, w = arr.shape
#make the image left-right
for j in range(0,w):
    for i in range(0,h):
        new_np_array[i,j] = arr[i,w-1-j]

In [ ]:
import pydicom
import os

In [ ]:
# Specify the directory containing the DICOM files
dataset_dir = '/content/drive/MyDrive/config/sample_data/10000879'

In [ ]:
dicom_files=[]
# Recursively search for DICOM files
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.dcm'):
            dicom_files.append(os.path.join(root, file))

In [ ]:
# Read each DICOM file and extract the pixel data
for file in dicom_files[:2]:
    dcm = pydicom.dcmread(file)
    pixel_data = dcm.pixel_array
    print(pixel_data)

In [ ]:
# Read in sample dicom file
ds = pydicom.dcmread(sample_file_name)
arr = ds.pixel_array

# Flip the dicom file left to right, the flipping logic is included in the dcm to png file conversion code
new_np_array = np.copy(arr)
#define the width(w) and height(h) of the image
h, w = arr.shape
#make the image left-right
for j in range(0,w):
    for i in range(0,h):
        new_np_array[i,j] = arr[i,w-1-j]

In [ ]:
# Getting the coordinates of ROI
rois = [int(s) for s in re.findall(r'\b\d+\b', sample_file_ROI)]
y1, x1, y2, x2 = rois

In [ ]:
# Display ROI on image
figure, ax = plt.subplots(1)
rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
ax.imshow(new_np_array, cmap="gray")
ax.add_patch(rect)

In [ ]:
import os
import pandas as pd

data = []

# Traverse the main directory
for root, dirs, files in os.walk('/content/drive/MyDrive/config/sample_data'):
    for file in files:
        if file.endswith('.dcm'):  # Modify the file extension if needed
            image_path = os.path.join(root, file)
            class_label = os.path.basename(os.path.dirname(image_path))
            data.append({'image_path': image_path, 'class': class_label})

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

In [ ]:
num_classes = len(df_merge['asses'].unique())
from keras.utils import to_categorical

# Convert labels to categorical format
df_merge['label'] = df_merge['asses'].astype('category').cat.codes
labels = to_categorical(df_merge['label'], num_classes)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_data, test_data, train_labels, test_labels = train_test_split(df_merge['anon_dicom_path'], labels, test_size=0.2, stratify=labels)


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Define the image size and other parameters
image_size = (224, 224)
batch_size = 32

# Define the data directory
data_directory = '/content/drive/MyDrive/config/sample_data'

# Load the dataframe containing image paths and labels
df_merge = pd.read_csv('/content/drive/MyDrive/config/tables/df_merge.csv')  # Replace with the actual path to your dataframe file

# Create the data generator
data_generator = ImageDataGenerator(rescale=1.0/255.0)


# Create data generator for image loading and preprocessing
datagen = ImageDataGenerator(rescale=1./255)

# Create data generator for image loading and preprocessing
train_data_gen = datagen.flow_from_dataframe(
   dataframe=df_merge,
   x_col='anon_dicom_path',  # Replace with the actual column name
   y_col='asses',
   target_size=image_size,
   classes=class_labels,
   class_mode='categorical',
   batch_size=batch_size,
   subset='training'
)

test_data_gen = datagen.flow_from_dataframe(
   dataframe=df_merge,
   x_col='anon_dicom_path',  # Replace with the actual column name
   y_col='asses',
   target_size=image_size,
   classes=class_labels,
   class_mode='categorical',
   batch_size=batch_size,
   subset='validation'
)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_data_gen, validation_data=test_data_gen, epochs=10)
